In [2]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from pathlib import Path

In [21]:
Datapath="/Users/samara/desktop/Experiments/"
Datapath = Path(Datapath)
arenas=9


masks = {}
Exps = [experiment for experiment in Datapath.glob("experiment*")]
for experiment in Exps:
    experiment_number = int(experiment.stem.replace("experiment", ""))
    print(f'Processing experiment: {experiment_number}')
    experiment_dir = f"mazes_experiment{experiment_number}"
    if os.path.exists(experiment_dir):
        shutil.rmtree(experiment_dir)
    os.makedirs(experiment_dir)

    # Find the first image in experiment
    images = [image for image in experiment.glob("*.jpg")]
    images.sort()
    image = images[0]
    print(f'Processing image: {image}')

    # Read the image
    img = cv2.imread(str(image), cv2.IMREAD_GRAYSCALE)
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Enter the arenas coordinates
    X1 = 0
    X2 = 710
    X3 = 1450
    X4 = 2200
    X5 = 2980
    X6 = 3690

    Y1 = 0
    Y2 = 725
    Y3 = 1140
    Y4 = 1860
    Y5 = 2350
    Y6 = 3200

     # Make tuples containing the 9 combinations of coordinates required to get the arenas
    regions_of_interest = [
        (X1, Y1, X2, Y2),
        (X3, Y1, X4, Y2),
        (X5, Y1, X6, Y2),
        (X1, Y3, X2, Y4),
        (X3, Y3, X4, Y4),
        (X5, Y3, X6, Y4),
        (X1, Y5, X2, Y6),
        (X3, Y5, X4, Y6),
        (X5, Y5, X6, Y6),
    ]

     # Dictionary to store the images, useful for the plot
    arena_images = {}
    
    for arena_number in range(arenas):
        print(f'Processing arena: {arena_number}')
        arena_dir = f"{experiment_dir}/arena{arena_number}"
        os.makedirs(arena_dir, exist_ok=True)

        # Crop the image
        arena = img[
            regions_of_interest[arena_number][1]:regions_of_interest[arena_number][3],
            regions_of_interest[arena_number][0]:regions_of_interest[arena_number][2],
        ]

        # Store the image in the dictionary
        arena_name = (f"experiment{experiment_number}_arena{arena_number}")
        arena_images[arena_name] = arena

        #To check arenas crop
        '''fig, ax = plt.subplots()
        plt.axis("off")
        im = ax.imshow(arena, cmap="gray", vmin=0, vmax = 255)
        plt.title(arena_name)
        plt.show()'''

        # Dictionary to store the images
        maze_images = {}

        #threshold definition to understand what to crop 
        _, binary = cv2.threshold(arena,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)

        #closing to remove black dots inside our kernel
        # Define the kernel for erosion
        kernel = np.ones((30, 30), np.uint8)  # Adjust kernel size as needed

        # Apply erosion
        closing = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

        # Apply connected components to label connected regions (so to extract the shape we are interested in) and get statistics
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(closing)

        for maze in range(1, num_labels):
            print(f'Processing maze: {maze}')
            maze_dir = f"{arena_dir}/maze{maze}"
            os.makedirs(maze_dir, exist_ok=True)

            # Get statistics for the component
            left, top, width, height, area = stats[maze]
            centroid_x, centroid_y = centroids[maze]

            # Create a mask for this specific component
            component_mask = np.uint8(labels == maze) * 255 

            # Use bitwise AND operation to extract the shape
            shape = cv2.bitwise_and(arena, arena, mask=component_mask)

            shape_crop= shape[top:top+height, left:left+width]
            maze_name = f"{arena_name}_maze{maze}"
            maze_images[maze_name] = shape_crop

            # Rotate the image
            if maze == 3:
                shape_crop = cv2.rotate(shape_crop, cv2.ROTATE_180)

            # store masks informations
            masks[maze_name] = shape_crop

            # print the mask
            '''fig, ax = plt.subplots()
            plt.axis("off")
            im = ax.imshow(shape_crop, cmap="gray", vmin=0, vmax=255)
            plt.title(maze_name)
            plt.show()'''

            #store Arena and masks in variables


Processing experiment: 1
Processing image: /Users/samara/desktop/Experiments/experiment1/image0.jpg
Processing arena: 0
Processing maze: 1
Processing maze: 2
Processing maze: 3
Processing arena: 1
Processing maze: 1
Processing maze: 2
Processing maze: 3
Processing arena: 2
Processing maze: 1
Processing maze: 2
Processing maze: 3
Processing arena: 3
Processing maze: 1
Processing maze: 2
Processing maze: 3
Processing arena: 4
Processing maze: 1
Processing maze: 2
Processing maze: 3
Processing arena: 5
Processing maze: 1
Processing maze: 2
Processing maze: 3
Processing arena: 6
Processing maze: 1
Processing maze: 2
Processing maze: 3
Processing arena: 7
Processing maze: 1
Processing maze: 2
Processing maze: 3
Processing arena: 8
Processing maze: 1
Processing maze: 2
Processing maze: 3
Processing experiment: 2
Processing image: /Users/samara/desktop/Experiments/experiment2/image0.jpg
Processing arena: 0
Processing maze: 1
Processing maze: 2
Processing maze: 3
Processing arena: 1
Processing

In [26]:
'''# Apply the mask to another image
Datapath="/Users/samara/desktop/Experiments/experiment1"
Datapath = Path(Datapath)

# read image 1 of the experiment1
image = Datapath / "image1.jpg"
img2 = cv2.imread(str(image), cv2.IMREAD_COLOR)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
# Convert the component_mask to the appropriate data type
component_mask = np.uint8(component_mask)
# Apply the mask to another image
masked_image = cv2.bitwise_and(img2, img2, mask=component_mask)
plt.imshow(masked_image)'''

error: OpenCV(4.9.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/core/src/arithm.cpp:230: error: (-215:Assertion failed) (mtype == CV_8U || mtype == CV_8S) && _mask.sameSize(*psrc1) in function 'binary_op'
